In [ ]:
!git clone https://github.com/raphael-sch/VELMA.git
%cd VELMA

Cloning into 'VELMA'...
remote: Enumerating objects: 137, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 137 (delta 14), reused 24 (delta 10), pack-reused 104 (from 1)
Receiving objects: 100% (137/137), 101.75 MiB | 8.24 MiB/s, done.
Resolving deltas: 100% (37/37), done.
Updating files: 100% (78/78), done.
/content/VELMA


In [ ]:
!pip install openai

In [ ]:
import openai
import requests
import requests
import pandas as pd
from PIL import Image
from io import BytesIO
import math
import html
import re
import cv2
import numpy as np
import imageio
import os
import csv
import time
import json
import ast

CALL DATASET TO GET NAVIGATION TEXT, PANOID AND START_HEADING

In [ ]:
# Path to your JSON file
json_file = '/content/VELMA/datasets/touchdown_unseen/data/dev.json'

routes = []

# Load the JSON file and split the content by newlines
with open(json_file, 'r') as file:
    content = file.read()  # Read the file content

    # Split by lines (assuming each line is a separate JSON object)
    json_objects = content.strip().split('\n')

    # Parse each JSON object separately
    for obj in json_objects:
        try:
            data = json.loads(obj)  # Parse each JSON object
            routes.append(data)    # Append the parsed data to the list
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON object: {e}")

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(routes)

# Print the DataFrame to check if everything is loaded correctly
print(df)

# If you need to access the length of 'id' for each row in the DataFrame:
df['id_length'] = df['id'].astype(str).apply(len)

# Print the DataFrame with the new column
unique_id_count = df['instructions_id'].nunique()
print(f"Number of unique IDs: {unique_id_count}")

    city  route_id                pre_pano       pre_heading  \
0    nyc      4754  wOd2XDb0Js4uUZB1UHEmjA  121.872246611971   
1    nyc      7046  aqXt8prWxtLqaEQ0jYiRRg  264.463047862252   
2    nyc      2112  JJf0XkoQciZCxTHEpcAQpw  244.488995003821   
3    nyc     11882  S3xnRu6DiGStxOnDGpWHiQ  27.7760044566822   
4    nyc     11229  muETW50Yu_HmaqF8zdbAnA  57.3646240080053   
..   ...       ...                     ...               ...   
795  nyc       954  V25zegZIuyAWUEOL2IXvHg  269.056696086243   
796  nyc     10711  XQHHEmvtX9FRig5J0plgQw  125.526212921869   
797  nyc      1925  r4kDRfQNNTzuWcGby-1-3Q  209.557677264822   
798  nyc      2349  4tnXHMwuyLIxgZzLGBvGEQ  222.795579308865   
799  nyc      3970  7pgoQV3e_mlT_TtIJWR2CA  311.416499086761   

              pre_pitch                                        pre_center  \
0    0.0887000690027548   {"x":0.4188118811881188,"y":0.5903225806451613}   
1     -12.5421981054401  {"x":0.2584158415841584,"y":0.22258064516129034}   


KeyError: 'id'

CALL 7 SHOT LEARNING IN CONTEXT

In [ ]:
# Read the example learning text file
sample_learning = '/content/7_shot_lmm.txt'
with open(sample_learning, 'r') as file:
    shot_learning = file.read().strip()


PERFORM THE TEXT TO SEND THE REQUEST TO OPEN AI

In [ ]:
def structure(navigation_instruction, api_key, shot_learning):
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    # Correctly format the content for the prompt
    payload = {
        "model": "gpt-4o",
        "messages": [
            {
                "role": "user",
                "content": (
                f"Convert this navigation instruction: '{navigation_instruction}' "
                "into a data structure you must have (n) nodes and (n-1) edges to build a graph,"
                "it is mandatory to start with a node (Starting point) and an edge (Forward), if there is not an spatial relationship,"
                "each node must have its corresponding edge, except for the last one, intersections node usually have actions edges"
                " if the node has no action, place (Forward). Uses just Forward, Left, Right for actions, and On the left,"
                "and On the Right for spatial relationships, head to the right or head to the left means Forward,"
                "landmarks with the word light, is represented by a intersection node,"
                "always use the following examples to get just the graph structure without any additional details: "
                f"'{shot_learning}'. Do not show the navigation instruction."
            )
            }
        ],
        "max_tokens": 300
    }

    # Make the API request
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

    # Handle the response
    if response.status_code == 200:
        response_data = response.json()
        if 'choices' in response_data and len(response_data['choices']) > 0:
            data_structure = response_data['choices'][0]['message']['content'].strip()
            return data_structure
        else:
            raise ValueError("Response does not contain valid choices data.")
    else:
        raise Exception(f"API request failed with status code {response.status_code}: {response.text}")


New approach

In [ ]:
def structure(navigation_instruction, api_key, shot_learning):
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    # Correctly format the content for the prompt
    payload = {
        "model": "gpt-4o",
        "messages": [
            {
                "role": "user",
                "content": (
                    f"Convert the given navigation instruction '{navigation_instruction}' into a graph data structure with (n) nodes and (n-1) edges. Follow these rules:"
                    "- Start with a node labeled (Starting Point) and an edge labeled (Forward) if no spatial relationship is given."
                    "- Each node must correspond to an edge, except for the target location."
                    "- Nodes labeled (Intersection) must be decision points, with edges labeled (Right), (Left), or (Forward). Only (Right) or (Left) correspond to edges originating from (Intersection) nodes."
                    "- Landmarks must include spatial relationships like (On the right), (On the left,) or (Forward)."
                    "- If a node has no explicit action, use (Forward)."
                    "- Phrases like (Head to the right) or (Head to the left) are interpreted as (Forward)."
                    "- Landmarks containing (light) or (junction) correspond to nodes labeled (Intersection)."
                    "- The target location must be the final node."
                    "- For instructions specifying an (x) number of intersections, include (x) nodes labeled (Intersection)."
                    "- Another intersection in instructions means label an (Intersection) node"
                    f"- Use this example format for the graph structure without additional details: '{shot_learning}'"
                    "Do not include the original navigation instruction in your output."

                    )
            }
        ],
        "max_tokens": 300
    }

    # Make the API request
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

    # Handle the response
    if response.status_code == 200:
        response_data = response.json()
        if 'choices' in response_data and len(response_data['choices']) > 0:
            data_structure = response_data['choices'][0]['message']['content'].strip()
            return data_structure
        else:
            raise ValueError("Response does not contain valid choices data.")
    else:
        raise Exception(f"API request failed with status code {response.status_code}: {response.text}")


Function to convert into a list

In [ ]:
# Convert all non-serializable types to standard Python types
def convert_to_standard_types(data):
    if isinstance(data, dict):
        return {key: convert_to_standard_types(value) for key, value in data.items()}
    elif isinstance(data, list):
        return [convert_to_standard_types(item) for item in data]
    elif isinstance(data, np.integer):  # Handle numpy integer
        return int(data)
    elif isinstance(data, np.floating):  # Handle numpy float
        return float(data)
    else:
        return data


def clean_and_convert(raw_data):
    # Clean the raw data by removing any surrounding characters
    raw_data = raw_data.strip().replace("```json", "").replace("```", "").replace("plaintext", "").replace("{{", "{").replace("}}", "}").replace("{{{", "{").replace("}}}", "}")

    # Step 1: Fix Nodes to ensure keys are quoted
    cleaned_nodes = re.sub(r'(\d+):', r'"\1":', raw_data)

    # Step 2: Fix Edges to ensure it is a list of dictionaries
    cleaned_edges = re.sub(
        r'\{(\d+),\s*(\d+),\s*"([^"]+)"\}',
        r'{"from": \1, "to": \2, "action": "\3"}',
        cleaned_nodes
    )

    # Step 3: Fix the Edges array
    cleaned_edges = re.sub(r'("Edges":)\s*\[\[', r'\1 [', cleaned_edges)
    cleaned_edges = re.sub(r'\]\]', r']', cleaned_edges)  # Remove extra closing bracket

    # Final wrap with a single curly brace to create a valid JSON object
    cleaned_data = "{" + cleaned_edges + "}"

    # Step 4: Convert to dictionary
    try:
        return json.loads(cleaned_data)
    except json.JSONDecodeError as e:

        return None


Extration of navigation instructions data from dataset

In [ ]:
import time
fixed =[508,
        705,
        786]

#iterator
#for i in range(500, 800):
for i in fixed:
    instruction = df['navigation_text'][i]
    Instance_id = df['id'][i]

    #Requesting to Chat GPT-4o###############################################################################
    api_key = 'your_API_key'
    data_structure = structure(instruction, api_key, shot_learning)

    #Convert graph into a list of directories###############################################################3
    converted_result = clean_and_convert(data_structure)
    if converted_result is not None:
        my_dictionary = {
          "Instance_id": Instance_id,
          "Navigation_instruction": instruction,
          "Graph": converted_result,
          }

            # Define the filename for the JSON file
        filename = '/content/dev_graph_lmm.json'

            # Check if the file exists
        if os.path.exists(filename):
                try:
                    # Load the existing content
                    with open(filename, 'r') as file:
                        content = file.read().strip()  # Read the content and strip whitespace
                        if content:  # Check if not empty
                            accumulated_data = json.loads(content)
                        else:
                            accumulated_data = []  # Initialize an empty list if the file is empty
                except json.JSONDecodeError:
                    print("Error decoding JSON. The file may be corrupted. Initializing a new list.")
                    accumulated_data = []  # Reset if there’s an error
        else:
                # If the file does not exist, initialize a new list
          accumulated_data = []

            # Append the new dictionary to the accumulated data
        accumulated_data.append(my_dictionary)

            # Convert accumulated_data to standard types
        accumulated_data = convert_to_standard_types(accumulated_data)

          # Save the updated data to the JSON file
        with open(filename, 'w') as file:
            json.dump(accumulated_data, file, indent=4)
    else:
        print(f"{i},")
    time.sleep(1)

In [ ]:
i

19